In [1]:
import sys
import ecdsa
import random
from sage import all_cmdline as sg
import gmpy2
from collections import namedtuple
from ecdsa import util
Signature = namedtuple("signature",("r","s"))

In [2]:
curve_order = int(ecdsa.BRAINPOOLP256r1.order)

def modular_inv(a,b):
    return pow(a,-1,b)

def make_matrix(msgs, sigs, B):
    m = len(msgs)
    matrix = sg.Matrix(sg.QQ, m + 2, m + 2)
    msgn, rn, sn = [msgs[-1], sigs[-1][0], sigs[-1][1]]
    rnsn_inv = rn * modular_inv(sn, curve_order)
    mnsn_inv = msgn * modular_inv(sn, curve_order)

    for i in range(0,m):
        matrix[i,i] = curve_order

    for i in range(0,m):
        x0=(sigs[i][0] * modular_inv(sigs[i][1], curve_order)) - rnsn_inv
        x1=(msgs[i] * modular_inv(sigs[i][1], curve_order)) - mnsn_inv
        matrix[m+0,i] = x0
        matrix[m+1,i] = x1
 
    matrix[m+0,i+1] = (int(2**B) / curve_order)
    matrix[m+0,i+2] = 0
    matrix[m+1,i+1] = 0
    matrix[m+1,i+2] = 2**B

    return matrix


def privkeys_from_reduced_matrix(msgs, sigs, matrix):
    keys=[]
    msgn, rn, sn = [msgs[-1], sigs[-1][0], sigs[-1][1]]
    for row in matrix:
        potential_nonce_diff = row[0]
        potential_priv_key = (sn * msgs[0]) - (sigs[0][1] * msgn) - (sigs[0][1] * sn * potential_nonce_diff)
    try:
        potential_priv_key *= modular_inv((rn * sigs[0][1]) - (sigs[0][0] * sn), curve_order)
        key = potential_priv_key % curve_order
        if key not in keys:
            keys.append(key)
    except Exception as e:
        print(str(e)+"\n")
        pass
    return keys

In [3]:
#%% 
import ecdsa
import ecdsa.curves
import os
from hashlib import shake_128


BANNER = """
WELCOME to the demo version of sig1337nature.
Our signature signing is faster than anyone!

In our demo you can request up to 69 signatures!

To show how certain we are in our construction, we even included a bug bounty program.
"""

FLAG = "FLAG"


def efficient_k(msg):
    # Make semi-deterministic to not exhaust the entropy pool too fast
    return int.from_bytes(
        shake_128(msg).digest(16) + os.urandom(16),
        "big"
    )

def sign_msg(priv_key,msg):
    if b"flag" in msg:
        print("No way, jose!")
        return

    sig = priv_key.sign(msg, k=efficient_k(msg))

    print("Signature (hex):", sig.hex())

In [10]:
privkey = ecdsa.SigningKey.generate(curve=ecdsa.curves.BRAINPOOLP256r1)
pubkey = privkey.get_verifying_key()

In [44]:
#!/usr/bin/env python
# Author Dario Clavijo 2020
# based on previous work:
# https://blog.trailofbits.com/2020/06/11/ecdsa-handle-with-care/
# https://www.youtube.com/watch?v=6ssTlSSIJQE

inital_msg = b"0"
msg = int.from_bytes(inital_msg,"big")
run_mode = "LLL"

#%%
def sign():
    sig = privkey.sign(inital_msg, k=efficient_k(inital_msg))
    return util.sigdecode_string(sig,ecdsa.curves.BRAINPOOLP256r1.order)

msgs,sigs = [],[]

for _ in range(10):
    msgs.append(msg)
    sigs.append(sign())

In [47]:
for B in range(130):
    matrix = make_matrix(msgs, sigs, B)

    if run_mode == "LLL":
        new_matrix = matrix.LLL(early_red=True, use_siegel=True)
        keys = privkeys_from_reduced_matrix(msgs, sigs, new_matrix)
    else:
        new_matrix = matrix.BKZ(early_red=True, use_siegel=True)
        keys = privkeys_from_reduced_matrix(msgs, sigs, new_matrix)
    
    if privkey.privkey.__dict__["secret_multiplier"] in keys:
        print(keys,B)

        

In [26]:
yubikey_fixed_prefix = random.randrange(2**176, curve_order)
 
nonces = [random.randrange(1, 2**176) + yubikey_fixed_prefix for i in range(6)]
nonces = [efficient_k(b"test") for _ in range(6)]

In [30]:
nonces[1]-nonces[0]

-267730574632063055490125425938742350488

In [35]:
int(ecdsa.curves.BRAINPOOLP256r1.order)

76884956397045344220809746629001649092737531784414529538755519063063536359079

In [41]:
2**255

57896044618658097711785492504343953926634992332820282019728792003956564819968

In [38]:
efficient_k(b"0")

44578392149148294369728459117041393789350492594136792250668922189614772293520